In [ ]:
!pip install datasets
!pip install evaluate

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
df_train = pd.read_json('HW04 Texts Holdout.json', delimiter='\t')
df_test = pd.read_json("texts_holdout.json")

In [ ]:
data = {'labels' : [], 'text' : []}

data['labels'].extend([0 for i in range(len(df_code.code))])
data['text'].extend(list(df_code.code))

data['labels'].extend([0 for i in range(len(text_samples))])
data['text'].extend(text_samples)

data['labels'].extend([1 for i in range(len(comments))])
data['text'].extend(comments)

df = pd.DataFrame(data)

In [ ]:
df

In [ ]:
# train_size was 0.8 while testing and is changed to 0.95 to train final model

from sklearn.model_selection import train_test_split
df_train, df_eval = train_test_split(df, train_size=0.95,stratify=df.labels, random_state=42)

In [ ]:
from datasets import Dataset, DatasetDict
raw_datasets = DatasetDict({
    "train": Dataset.from_pandas(df_train),
    "eval": Dataset.from_pandas(df_eval)
})

In [ ]:
from transformers import  AutoModelForSequenceClassification, AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
classifier = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2).to('cuda')
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
tokenized_datasets = raw_datasets.map(lambda dataset: tokenizer(dataset['text'], truncation=True, padding='max_length', max_length=512, return_tensors='pt').to('cuda'), batched=True)

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(["text", "__index_level_0__"])
print(tokenized_datasets)

In [ ]:
from transformers import DataCollatorWithPadding, TrainingArguments, Trainer
import numpy as np
import evaluate

# Padding for batch of data that will be fed into model for training
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Training args
training_args = TrainingArguments("test-trainer", num_train_epochs=1, evaluation_strategy="epoch",
                                  weight_decay=5e-4, save_strategy="no", report_to="none")

# Metric for validation error
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc") # F1 and Accuracy
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Define trainer
trainer = Trainer(
    classifier,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["eval"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()